In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import kapre
import keras
import os
import soundfile as sf
import time
import progressbar
import gc
import h5py

Using TensorFlow backend.


In [2]:
# Load the training and validation data from hdf5
training = h5py.File('/data/musicnet/hdf5/train_data.hdf5', 'r')
validation = h5py.File('/data/musicnet/hdf5/val_data.hdf5', 'r')

list(training.keys())
training_data = training['data']
training_targets = training['targets']

list(validation.keys())
validation_data = validation['data']
validation_targets = validation['targets']

In [ ]:
# LSTM AUTOENCODER

In [3]:
# Reshaping the data into sequences
# The shape needs to be (number of sample, number of timesteps, number of features)
# In this case we have a sequence of frequencies, i.e. just one feature, with 88200 (sampling rate of 44100 per
# second over 2 second chunks)
train_data = training_data.value.reshape(training_data.shape[0],88200,1)
val_data = validation_data.value.reshape(validation_data.shape[0],88200,1)
print(train_data.shape)
print(val_data.shape)

(39472, 88200, 1)
(8089, 88200, 1)


In [29]:
# Defining the model
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

inputs = Input(shape=(88200, 1))
encoded = LSTM(128)(inputs)

decoded = RepeatVector(88200)(encoded)
decoded = LSTM(1, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [30]:
sequence_autoencoder.compile(loss='mean_squared_error', optimizer='adam')  # original

In [32]:
sequence_autoencoder.fit(train_data[:20], train_data[:20],
                     epochs=1,
                     batch_size=10,
                     shuffle=True,
                     )

Epoch 1/1
20/20 [==============================] - 267s 13s/step - loss: 0.0052
